## VRP SCHEDULING

In [1]:
from __future__ import print_function
from ortools.linear_solver import pywraplp

import numpy as np
import scipy as sp
from numpy import matrix

## Input data

In [102]:
## TEST DATA

# load dispatsh matrix - demand
    #LJ, CE, MB
D = [17, 6, 25]

# vehicle load capacity
C = [100, 100]

#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,CE,MB
E = [
    [10, 0, 11],
    [0, 21, 26],
    [31, 32, 0],]

#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]
#-E (negative values)
negE = np.negative(E)

#number of nodes
N = np.size(E,0)
#number of edges
M = np.size(E,1)
#number of vehicles
V = np.size(C)

#[0, 0, 1, 1, 0, 0]
E

[[10, 0, 11], [0, 21, 26], [31, 32, 0]]

In [73]:

# load dispatsh matrix - demand
    #LJ, CE, MB, KP, NM
D = [17, 6, 25, 37, 12]

# vehicle load capacity
C = [100, 100]

#network graph, stopci = povezave 12, 23, 34, 45, 35, 13, vrstice = mesta
    #LJ,CE,MB,KP,NM
E = [
    [10, 0, 0, 0, 16, 11],
    [0, 21, 23, 0, 0, 26],
    [31, 32, 0, 0, 0, 0],
    [0, 0, 0, 44, 45, 0],
    [0, 0, 53, 54, 0, 0]]

#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]
#-E (negative values)
negE=np.negative(E)

#number of nodes
N = np.size(E,0)
#number of edges
M = np.size(E,1)
#number of vehicles
V = np.size(C)

#[0, 0, 1, 1, 0, 0]
#E

## Creating matrix A

In [134]:
#creating A1 matrix - constraint IV, graph data 
#number of rows = number of columns (E) x number of cycles * 2 (for negative and positive notation);
#columns = [number of columns (E) x number of cycles] + [number of rows (E) x number of cycles]
# create zero A1 matrix

A1 = np.zeros((len(Et)*V*2, len(E)*V+len(Et)*V*2))
for j in range (0, len(Et)):
    for k in range (0, V):
        for i in range (0, len(E)):
            A1[j*V+k][k*len(E)+i]=Et[j][i]
            #print (j*V+k, k*len(E)+i, j, i) #=np.negative(Et[j][i])
for j in range (0, len(Et)):
    for k in range (0, V):
        for i in range (0, len(E)):
            A1[len(Et)*V+j*V+k][k*len(E)+i]=np.negative(Et[j][i])
            #print ( len(Et)*V+ j*V+k, k*len(E)+i, j, i) #=np.negative(Et[j][i])
            
# Adding coeficients for slack variables 
for j in range (0, len(Et)*V): 
    A1[j][len(E)*V+j] = -2
for j in range (0, len(Et)*V): 
    A1 [len(Et)*V+j][len(Et)*V+len(E)*V+j] = 2
#A1

In [135]:
# creating matrix A2 - II constraint, the number of packets on the node is all patckets dispatched
#number of rows: 2 * V + 2* Et = 3 * V
#number of columns: Et * V
A2 = np.zeros((2*V+2*len(Et), (len(Et)*V))) 
for j in range (0, V):
    for i in range (j*len(Et), (j+1)*len(Et)):
        #print("value is", j, i)
        A2[j,i]=1
for j in range (0, V):
    for i in range (j*len(Et), (j+1)*len(Et)):
        #print("value is", j+V, i)
        A2[V+j,i]=-1       
for j in range (0, V*len(Et)):
        #print("value is", 2*V+j, j)
        A2[2*V+j,j]=-1

In [136]:
#concatenate A1 and A2 = A matrix
A1extend = np.c_[A1, np.zeros((len(A1), len(A2[0])))]
A2extend = np.c_[np.zeros((len(A2), len(A1[0]))), A2]

A=A1extend.copy()
for i in range (0, len(A2)):
    A=np.vstack([A, A2extend[i,:]])

## Declaring coeficients for constraints - right side of equation

In [10]:
# coeficients for A1 matrix = 0
b1=[]
for i in range (0, len(A1extend)):
    b1.append(0) 

# coeficients for A2 matrix
b2 = D + np.negative(D)
for i in range(0, 2*len(Et)):
    b2.append(0)
    
# consolidated vector of coeficients
b = b1 + b2

## Creating variables

In [159]:
# CREATE VARIABLES  X - vector with c11-cnn variables
# vsak vektor v matriki X1n = dolžine len(E). 
#Skupno število vseh vektorjev X1n = len (Et)
#X1 - variables on objective function 
#K - slack variables
#Ow - cycles load dispatch variables

X1size = len(E) * V   # number of all variables
X1 = []
for i in range (0, X1size):
    var='x'+str(i)
    X1.append(var)
 
#creating vector with K variables
K = []
for j in range (0, len(A1)):
    #creating vector with K variables
    var='k'+str(j)
    K.append(var)
    
#creating vector with O variables
Ow = []
for j in range(0, V*len(Et)):
    var='Ow'+str(j)
    Ow.append(var)
    
#Final X vector vith all variables
X=X1.copy()
for i in range (0,len(K)):
    X.append(K[i])
    #print (K[i])

for j in range (0,len(Ow)):
    X.append(Ow[j])
print("number of bariables =", len(X))

number of bariables = 24


In [160]:
## Declaring variables to the solver

#Declaring the solver
solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x_min = 0.0                                # lower variables border 
x_max = solver.infinity()                  # Upper variables border                        
#X_int=X1
#X_real=K
#def declare_variables(X_int, X_real,x_min, x_max):

variables = [] 

for varN, xi_name in enumerate(X1):                          # declaring objective variables 
    variables.append(solver.IntVar(x_min, x_max, xi_name))
    
for varN, xi_name in enumerate(K):                          # declaring slack variables
    variables.append(solver.NumVar(x_min, x_max, xi_name))

for varN, xi_name in enumerate(Ow):                          # declaring load doispatch variables
    variables.append(solver.IntVar(x_min, x_max, xi_name))

print('Number of variables created =', solver.NumVariables())
    #for variable in variables:
        #print('%s = %d' % (variable.name(), variable.solution_value()))

Number of variables created = 24


## Declare constraints

In [161]:

b = np.zeros((1, len(A)))           # coeficients on the right side of constraint equation

for rowN, row in enumerate(A):
    left_side = None
    for colN, coeff in enumerate(row):
        if coeff == 0:
            continue
        if left_side is None:
            left_side = coeff*variables[colN]
        else:
            left_side += coeff*variables[colN]
    if left_side is None and b[0,rowN] < 0:
        raise ValueError('Constraint ' + str(rowN) + ' cannot be satisfied!')
    if left_side is not None:
        solver.Add(left_side <= b[0,rowN])
        
print('Number of constraints added =', solver.NumConstraints())

Number of constraints added = 22


## Declare Objective Function

In [162]:
# declare objective function: x0+x1+x2+x3+x4+......

#C1 = [variables[0], variables[1], variables[2], variables[3], variables[4], variables[5]]
C = []
for i in range (0, len(X1)):
    C.append(variables[i])
objective = sum (C)
solver.Minimize(objective)

# Weights for prioritizing different variables
#wgt_X1 = 1.0
#X1_coeffs = [ wgt_X1 for _ in X1 ]
#for coeffN, coeff in enumerate(X1_coeffs):
#    if coeff != 0:
#        C += coeff*X1[coeffN]

        
#print(C)
#print(sum(C))

#solver.Maximize(x0 + x1 + x2 + x3 + x4 + x5 + x6 +x7 + x8 + x9)   

## Invoke the solver

In [163]:
result_status = solver.Solve()
    # The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

In [164]:
assert solver.VerifySolution(1e-7, True)

print('Solution:')
print('Objective value =', solver.Objective().Value())
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('x0 =', variables[0].solution_value())
print('x1 =', variables[1].solution_value())
print('x2 =', variables[2].solution_value())
print('x3 =', variables[3].solution_value())
print('x4 =', variables[4].solution_value())
print('x5 =', variables[5].solution_value())
print('x6 =', variables[6].solution_value())
print('x7 =', variables[7].solution_value())
print('x8 =', variables[8].solution_value())
print('x9 =', variables[9].solution_value())

print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())

Solution:
Objective value = 0.0
Number of variables = 24
Number of constraints = 22
x0 = 0.0
x1 = 0.0
x2 = 0.0
x3 = 0.0
x4 = 0.0
x5 = 0.0
x6 = 0.0
x7 = 0.0
x8 = 0.0
x9 = 0.0

Advanced usage:
Problem solved in 92839.000000 milliseconds
Problem solved in 0 iterations
Problem solved in 0 branch-and-bound nodes
